In [1]:
from datetime import datetime

import ee
import geopandas as gpd
import pandas as pd

from fwi_predict.geo.ee import get_gfs

In [2]:
# Initialize Earth Engine
ee.Initialize()

In [18]:
# Read pond metadata
ponds = gpd.read_file("../data/clean/pond_metadata_clean.geojson")

# Get first pond coordinate to use as representative point
first_lon = ponds.geometry.iloc[0].x
first_lat = ponds.geometry.iloc[0].y
point = ee.Geometry.Point(first_lon, first_lat)

In [19]:
# Get GFS collection
gfs = get_gfs()

In [20]:
point

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Point",
    "arguments": {
      "coordinates": {
        "constantValue": [
          81.10742294224373,
          16.63587019484362
        ]
      }
    }
  }
})

In [29]:
# Get most recent forecast date
tomorrow = ee.Date(datetime.now()).advance(1, 'day')
latest_forecast = gfs \
		.filterDate(tomorrow.advance(-15, 'day'), tomorrow) \
		.filterBounds(point) \
    .sort('system:time_start', False) \
    .first()
forecast_date = datetime.fromtimestamp(
    latest_forecast.get('system:time_start').getInfo()/1000
).astimezone()

print(f"Most recent GFS forecast available is from: {forecast_date}")


Most recent GFS forecast available is from: 2025-03-30 08:00:00-04:00
